## Find the area of the Urban Residents neighborhood

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

# Load the datasets
neighborhoods = gpd.read_file("../../data/raw/neighborhoods.geojson")

urban_polygon = neighborhoods.loc[neighborhoods["name"] == "Urban Residents"]

urban_polygon.head()

In [ ]:
# Create a copy of the urban_polygon GeoDataFrame and convert it to Web Mercator EPSG:3857
urban_polygon_3857 = urban_polygon.to_crs(epsg=3857)
urban_polygon_3857.head()

In [ ]:
# Print the area of urban_polygon_3857 in kilometers squared
area_km2 = urban_polygon_3857.geometry.area / 1e6
print(f"Area of Urban Residents neighborhood: {area_km2.values[0]:.2f} km squared")

## The center of the Urban Residents neighborhood

In [ ]:
# Create downtown_center from urban_poly_3857's geometry centroid
downtown_center = urban_polygon_3857.geometry.centroid
downtown_center

In [ ]:
# Plot the urban_polygon_3857 and downtown_center
ax = urban_polygon_3857.plot(color="lightgreen", alpha=0.5, figsize=(10, 10))
downtown_center.plot(
    ax=ax, color="red", marker="X", markersize=100, label="Downtown Center"
)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=urban_polygon_3857.crs)
ax.set_axis_off()
ax.set_title("Urban Residents Neighborhood with Downtown Center")
ax.legend()
plt.show()

## การเตรียมข้อมูลเพื่อคำนวณระยะทาง

ในแบบฝึกหัดนี้ คุณจะเตรียม GeoDataFrame ชื่อ `art_dist_meters`  
โดยแปลงพิกัดตำแหน่งผลงานศิลปะในย่านดาวน์ทาวน์ให้อยู่ในหน่วยเมตร  
ด้วยระบบพิกัด **EPSG:3857**

จากนั้น ในแบบฝึกหัดถัดไป คุณจะใช้ `art_dist_meters`  
เพื่อคำนวณระยะทางของผลงานศิลปะแต่ละชิ้นจากจุดศูนย์กลางของย่าน **Urban Residents** ในหน่วยเมตร

In [ ]:
import pandas as pd
from shapely.geometry import Point

# Create a dataframe of art locations
art_df = pd.read_csv("../../data/raw/public_art.csv")
art_dist_meters = gpd.GeoDataFrame(
    art_df,
    geometry=gpd.points_from_xy(art_df["Longitude"], art_df["Latitude"]),
    crs="EPSG:4326",
)
art_dist_meters.head(2)

In [ ]:
# Set the coordinate reference system to Web Mercator (EPSG:3857) for distance calculations
art_dist_meters_3857 = art_dist_meters.to_crs(epsg=3857)
art_dist_meters_3857.head(2)

In [ ]:
center_point = downtown_center.iloc[0]
center_point

# Add a new column to art_dist_meters_3857 that calculates the distance from each art location to the downtown center
art_dist_meters_3857["center"] = center_point

## การคำนวณระยะทางของผลงานศิลปะจากศูนย์กลางย่าน

ตอนนี้คุณมีทั้งจุดศูนย์กลางของย่าน และตำแหน่งผลงานศิลปะที่ถูกแปลงพิกัดให้อยู่ในหน่วยเมตรเรียบร้อยแล้ว  
ขั้นตอนถัดไปคือการคำนวณระยะทางของผลงานศิลปะแต่ละชิ้นจากจุดศูนย์กลางของย่านในหน่วยเมตร

In [ ]:
# Import package for pretty printing
import pprint

# Build a dictionary of titles and distances for Urban Residents art
art_distances = {}
for row in art_dist_meters_3857.iterrows():
    vals = row[1]
    key = vals["Title"]
    ctr = vals["center"]
    art_distances[key] = vals["geometry"].distance(ctr)

# Pretty print the art_distances
pprint.pprint(art_distances)

## สร้างตำแหน่งสำหรับ Folium จากจุดศูนย์กลางของย่าน

เพื่อสร้างแผนที่ด้วย Folium สำหรับย่าน **Urban Residents**  
คุณต้องสร้างค่าพิกัดตำแหน่ง (coordinate pair)  
ให้อยู่ในรูปแบบที่ Folium รองรับ

In [ ]:
urban_polygon.head()

In [ ]:
urban_polygon.centroid

In [ ]:
# Create urban_center
urban_center = urban_polygon.centroid.squeeze()
urban_center

In [ ]:

# Create array for folium map (latitude, longitude)
urban_location = [urban_center.y, urban_center.x]
urban_location

## Create a folium map of downtown Nashville

In [ ]:
# Construct a folium map with urban_location
import folium

downtown_map = folium.Map(location=urban_location, zoom_start=15)

# Display the map
display(downtown_map)

In [ ]:
# Create array for called folium_loc from the urban_polygon center point
point = urban_polygon.centroid.squeeze()
folium_loc = [point.y, point.x]

# Construct a map from folium_loc: downtown_map
downtown_map = folium.Map(location = folium_loc, zoom_start = 15)

# Draw our neighborhood: Urban Residents
folium.GeoJson(urban_polygon.geometry).add_to(downtown_map)

# Display the map
display(downtown_map)

In [ ]:
art_gdf = gpd.GeoDataFrame(art_df, geometry=gpd.points_from_xy(art_df["Longitude"], art_df["Latitude"]), crs="EPSG:4326")
neighborhoods_with_art = gpd.sjoin(art_gdf, neighborhoods, predicate='within')
urban_art = neighborhoods_with_art[neighborhoods_with_art["name"] == "Urban Residents"]

In [ ]:
urban_art.head(2)

In [ ]:

# Iterate through the urban_art and print each part of tuple returned
for row in urban_art.iterrows():
  print('first part: ', row[0])
  print('second part: ', row[1])

# Create a location and marker with each iteration for the downtown_map
for row in urban_art.iterrows():
    row_values = row[1] 
    location = [row_values['Latitude'], row_values['Longitude']]
    marker = folium.Marker(location = location)
    marker.add_to(downtown_map)

# Display the map
display(downtown_map)

In [ ]:
# Print the urban_art titles
print(urban_art['Title'])

#Print the urban_art descriptions
print(urban_art['Description'])

# Replace Nan and ' values in description
urban_art['Description'].fillna('', inplace = True)
urban_art['Description'] = urban_art['Description'].str.replace("'", "`")

#Print the urban_art descriptions again
print(urban_art['Description'])

In [ ]:
# Construct downtown map
downtown_map = folium.Map(location = urban_location, zoom_start = 15)
folium.GeoJson(urban_polygon).add_to(downtown_map)

# Create popups inside the loop you built to create the markers
for row in urban_art.iterrows():
    row_values = row[1] 
    location = [row_values['Latitude'], row_values['Longitude']]
    popup = (str(row_values['Title']) + ': ' + 
             str(row_values['Description'])).replace("'", "`")
    marker = folium.Marker(location = location, popup = popup)
    marker.add_to(downtown_map)

# Display the map.
display(downtown_map)